In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_parquet("hf://datasets/shahules786/PoetryFoundationData/data/train-00000-of-00001-486832872ed96d17.parquet")
print(f"\n\n======\n\n")

print(df.columns)

newyork = df[df['author'].isin(["John Ashbery", "Barbara Guest", "James Schuyler", "Kenneth Koch", "Frank O'Hara"])]
shake = df[df['author'] == 'William Shakespeare']

print(f"Shakespeare: {len(shake)} examples\nNew Yorkers: {len(newyork)} examples")
print(f"Shakespeare avg length: {np.average([len(poem) for poem in shake['content']])}\nNew Yorkers avg length: {np.average([len(poem) for poem in newyork['content']])}")

C:\Users\Anthony\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm






Index(['poem name', 'content', 'author', 'type', 'age'], dtype='object')
Shakespeare: 85 examples
New Yorkers: 81 examples
Shakespeare avg length: 1468.5058823529412
New Yorkers avg length: 1810.6049382716049


In [3]:
def load_embedding_model():
    """ Load GloVe Vectors
        Return:
            wv_from_bin: All 400000 embeddings, each length 50
    """
    import gensim.downloader as api
    wv_from_bin = api.load("glove-wiki-gigaword-50")
    # wv_from_bin = api.load("glove.6B/glove.6B.50d.txt")
    print("Loaded vocab size %i" % len(list(wv_from_bin.index_to_key)))
    return wv_from_bin
wv_from_bin = load_embedding_model()

Loaded vocab size 400000


In [4]:
def process_poem_into_list_of_words(poem) :
  out = poem.replace('\n', ' ')
  out = out.lower()
  out = re.sub(r'[^a-zA-Z ]+', '', out)
  return [word for word in out.split(' ') if word!='']

newyork_processed = [process_poem_into_list_of_words(newyork['content'].iloc[i]) for i in range(len(newyork))]
newyork_labels = [0 for i in range(len(newyork))]
shake_processed = [process_poem_into_list_of_words(shake['content'].iloc[i]) for i in range(len(shake))]
shake_labels = [1 for i in range(len(shake))]

processed_poems = newyork_processed + shake_processed
labels = newyork_labels + shake_labels
# perm = np.random.permutation(len(processed_poems))
# processed_poems = processed_poems[perm]
# labels = labels[perm]

vocab = sorted(list(set([word for poem in processed_poems for word in poem])))

word_to_idx = {word:idx for idx, word in enumerate(vocab)}
idx_to_word = {idx:word for idx, word in enumerate(vocab)}

for poem in processed_poems :
  for i in range(len(poem)) :
    poem[i] = word_to_idx[poem[i]]

print(processed_poems[0])

embedding_matrix = np.zeros((len(vocab), 50))
bad_count = 0
for i, word in enumerate(vocab):
    try:
        embedding_matrix[i] = wv_from_bin.get_vector(word)
    except:
        print("this is bad", word)
        bad_count += 1
print(f"Total Bad Words: {bad_count} out of total vocab {len(vocab)}")

[3916, 277, 1130, 5168, 2838, 5188, 5137, 7516, 4134, 8039, 2896, 6310, 5526, 4076, 3548, 318, 5514, 4891, 1130, 2305, 3292, 56, 1549, 7151, 795, 2619, 351, 7546, 1442, 8442, 0, 6303, 386, 2551, 4242, 558, 7529, 3884, 2557, 8358, 3430, 3333, 2384, 7508, 8582, 4980, 4777, 7508, 8582, 238, 7546, 1374, 5137, 4277, 6393, 4695, 8442, 1754, 7516, 1740, 5520, 5220, 7255, 5516, 5107, 4092, 1267, 5713, 5107, 1268, 5073, 7542, 318, 1470, 7670, 4875, 8115, 5107, 224, 981, 7516, 4004, 3916, 2309, 7571, 4781, 351, 3692, 8202, 5188, 5107, 8589, 6249, 117, 867, 1663, 8442, 455, 238, 2932, 3135, 455, 3884, 4271, 2932, 3884, 7946, 4271, 8242, 3925, 5185, 2084, 238, 8242, 3925, 7516, 4561, 7516, 4423, 5107, 4260, 5163, 5107, 4331, 8286, 8325, 4593, 1581, 0, 4769, 6904, 7670, 541, 2910, 3766, 1660, 3122, 5107, 4271, 1535, 2127, 6559, 3766, 7571, 4781, 7510, 3409, 6485, 8049, 122, 3692, 4089, 7510, 3692, 844, 7695, 4833, 5137, 4875, 5238, 6850, 5399, 5107, 7561, 351, 7546, 1374, 7670, 4577, 7546, 318, 572

In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_length = max(len(poem) for poem in processed_poems)
print(f"Max Length = {max_length}")
padded_poems = pad_sequences(processed_poems, maxlen=max_length, padding='post', truncating='post')

Max Length = 9713


In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding

model = tf.keras.Sequential()
e = Embedding(len(vocab), 50, weights=[embedding_matrix], input_length = max_length, trainable=False)
model.add(e)
model.add(LSTM(100, input_shape = (max_length, 50)))
model.add(Dense(1, activation='sigmoid'))
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(0.1, decay_rate=0.1, decay_steps=1000)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
print(model.summary())

C:\Users\Anthony\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
C:\Users\Anthony\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │       430,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 430,100 (1.64 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 430,100 (1.64 MB)

None


In [7]:
perm = np.random.permutation(len(padded_poems))
shuffled_poems = np.array(padded_poems)[perm]
shuffled_labels = np.array(labels)[perm]

training_data = shuffled_poems[:-20]
training_labels = shuffled_labels[:-20]

validation_data = shuffled_poems[-20:]
validation_labels = shuffled_labels[-20:]

In [12]:
model.fit(np.array(training_data), np.array(training_labels), epochs=20, verbose=1)

Epoch 1/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 53s 10s/step - accuracy: 0.4765 - loss: 2.2703
Epoch 2/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 86s 17s/step - accuracy: 0.4938 - loss: 1.6296
Epoch 3/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 80s 16s/step - accuracy: 0.4997 - loss: 0.8831
Epoch 4/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 87s 17s/step - accuracy: 0.5071 - loss: 0.7730
Epoch 5/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 92s 18s/step - accuracy: 0.4732 - loss: 0.8412
Epoch 6/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 83s 16s/step - accuracy: 0.5112 - loss: 0.7378
Epoch 7/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 88s 17s/step - accuracy: 0.4764 - loss: 0.7208
Epoch 8/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 84s 16s/step - accuracy: 0.5305 - loss: 0.7112
Epoch 9/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 102s 21s/step - accuracy: 0.4927 - loss: 0.6956
Epoch 10/20
1/5 ━━━━━━━━━━━━━━━━━━━━ 1:32 23s/step - accuracy: 0.4062 - loss: 0.7001

KeyboardInterrupt: 

In [ ]:
loss, accuracy = model.evaluate(np.array(validation_data), np.array(validation_labels), verbose=1)

In [18]:
print(loss, accuracy)

0.0004358454898465425 1.0
